# Analizy danych za pomocą biblioteki *revoscalepy* z wykorzystaniem wewnętrznego kontekstu serwera SQL 

https://docs.microsoft.com/pl-pl/r-server/python-reference/revoscalepy/rxinsqlserver

### Informacje o środowisku wykonawczym

- Stosowana metoda: **SQL Server Machine Learning Services (In-Database)**

In [1]:
import sys
import revoscalepy


Express Edition will continue to be enforced.


In [2]:
print('Wersja modułu revoscalepy: ', revoscalepy.__version__)
print('Ścieżka do biblioteki revoscalepy:  ', revoscalepy.__path__)
print('Wersja języka Python: ', sys.version)
print('Ścieżka do interpretera: ', sys.executable)

Wersja modułu revoscalepy:  9.2.0
Ścieżka do biblioteki revoscalepy:   ['D:\\Microsoft SQL Server\\MSSQL14.MSSQLSERVER\\PYTHON_SERVICES\\lib\\site-packages\\revoscalepy']
Wersja języka Python:  3.5.2 |Continuum Analytics, Inc.| (default, Jul  5 2016, 11:41:13) [MSC v.1900 64 bit (AMD64)]
Ścieżka do interpretera:  D:\Microsoft SQL Server\MSSQL14.MSSQLSERVER\PYTHON_SERVICES\python.exe


### Chwila na ustawienia

In [3]:
del revoscalepy #małe czyszczenie

In [4]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd

### Utworzenie kontekstu obliczeniowego wewnątrz serwera MS SQL 2017
- RxInSqlServer - klasa tworząca kontekst obliczeniowy na serwerze SQL
- RxSqlServerData - klasa realizująca dostęp do zbioru danych określonych przez zapytanie i zlokalizowanych na serwerze SQL

### Utworzenie kontekstu obliczeniowego - rozwiązanie

In [5]:
from revoscalepy import RxSqlServerData, RxInSqlServer, rx_lin_mod

In [6]:
connection_string="Driver=SQL Server;Server=.;Database=RevoTestDB;Trusted_Connection=True"

In [7]:
cc = RxInSqlServer(
    connection_string = connection_string,
    num_tasks = 1,
    auto_cleanup = False,
    console_output = True,
    execution_timeout_seconds = 0,
    wait = True
    )

In [8]:
type(cc)

revoscalepy.computecontext.RxInSqlServer.RxInSqlServer

### Analizowane dane

Pochodzenie:
http://stat-computing.org/dataexpo/2009/the-data.html <br>
Wiecej danych testowych jest dostępnych po instalacji narzędzi dla języka R, w lokalizacji dla przykładu: <br>**"D:\Microsoft SQL Server\MSSQL14.MSSQLSERVER\R_SERVICES\library\RevoScaleR\SampleData"** 

#### Opis pól występujących w tabeli
- **ArrDelay** - opóźnienie przylotu w minutach
- **CRSDepTime** - planowany czas wylotu (czas lokalny, hhmm)
- **DayOfWeek** - dzień 1 (Monday) - 7 (Sunday)

**query="select top 600000 [ArrDelay],[CRSDepTime],[DayOfWeek] FROM airlinedemosmall"**

In [9]:
data_source = RxSqlServerData(
    sql_query = "select top 600000 * from airlinedemosmall",
    connection_string = connection_string,
    column_info = {
        "ArrDelay" : { "type" : "integer" },
        "DayOfWeek" : {
            "type" : "factor",
            "levels" : [ "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday" ]
        }
    })

In [10]:
type(data_source)

revoscalepy.datasource.RxSqlData.RxSqlServerData

### Pobranie danych z bazy

In [11]:
data_source.head(num_rows = 6, report_progress = 3)

ReadNum=1, StartRowNum=1, CurrentNumRows=6, TotalRowsProcessed=6, ReadTime=0.05, ProcessDataTime = 0.001, LoopTime = 0.051 

Overall compute summaries time: 0.051 secs.
Total loop time: 0.051
Total read time for 1 reads: 0.05
Total process data time: Less than .001 seconds
Average read time per read: 0.05
Average process data time per read: Less than .001 seconds
Number of threads used: 2


,ArrDelay,CRSDepTime,DayOfWeek
0,6,9.666666,Monday
1,-8,19.916666,Monday
2,-2,13.750000,Monday
3,1,11.750000,Monday
4,-2,6.416667,Monday
5,-14,13.833333,Monday


### Pobranie podstawowych informacji o obiekcie *revoscalepy data source*

In [ ]:
from revoscalepy import rx_get_info, rx_get_var_info, \
                        rx_get_var_names, rx_summary, \
                        rx_list_keys, rx_read_object, \
                        rx_get_compute_context, rx_set_compute_context

### Ustawienie kontekstu lokalnego

In [ ]:
from revoscalepy import RxLocalSeq

In [ ]:
local_cc = RxLocalSeq()

In [ ]:
print(rx_get_compute_context())

### Ustawienie kontekstu na serwerze SQL

In [ ]:
#sql_cc = rx_set_compute_context(cc)
#print(rx_get_compute_context())

In [ ]:
info = rx_get_info(data_source)
print(info)

In [ ]:
var_info = rx_get_var_info(data_source, compute_info=True)
print(var_info)

In [ ]:
#rx_set_compute_context(local_cc)
name_info = rx_get_var_names(data_source)
print(name_info)

In [ ]:
info = rx_get_info(data_source, get_var_info=True)
print(info)

In [ ]:
summary = rx_summary("ArrDelay+DayOfWeek", data_source)
print(summary)

In [ ]:
keys = rx_list_keys(data_source)
print(type(keys))
print(len(keys))

In [ ]:
print(keys['DayOfWeek'][:10])

### Dane z obiektu *data_source* można zaimportować do obiektu *DataFrame* z biblioteki *Pandas* 

In [ ]:
from revoscalepy import rx_import

In [ ]:
df = pd.DataFrame(rx_import(input_data = data_source))

In [ ]:
print(type(df))

In [ ]:
df.sample(n=5)

In [ ]:
df.sort_values(by='ArrDelay', ascending=False)[:10]

In [ ]:
df.pivot_table(columns='DayOfWeek')

Wynik w każdym z wierszy reprezentuje wartość średnią dla poszczególnego dnia tygodnia

In [ ]:
df.groupby('DayOfWeek')[df['DayOfWeek'] == 'Monday'].hist(column='CRSDepTime', figsize=[8,6])

In [ ]:
df['delayed'] = df['ArrDelay'].apply(lambda x: x > 0)

In [ ]:
df.sample(n=5)

In [ ]:
df['delayed'].value_counts()

In [ ]:
not_delayed = df['delayed'].value_counts()[0]
print('Ilość lotów bez opóźnienia:', not_delayed)

In [ ]:
delayed = df['delayed'].value_counts()[1]
print('Ilość lotów opóźnionych:', delayed)

In [ ]:
total_flights=not_delayed+delayed
print('Małe sprawdzenie:', total_flights)

In [ ]:
print('Współczynnik samolotów przylatujących na czas:', float(not_delayed) / total_flights)

In [ ]:
group_by_dayOfWeek = df.groupby(['DayOfWeek','delayed'])
print(group_by_dayOfWeek)
print(group_by_dayOfWeek.size())

In [ ]:
delays_by_dayOfWeek = group_by_dayOfWeek.size().unstack()
delays_by_dayOfWeek

In [ ]:
delays_by_dayOfWeek.plot(kind = 'barh', 
                               stacked = True, 
                               figsize=[14,6], 
                               colormap='autumn',
                               grid = True,
                               rot = 45,
                               title='Relacja samolotów opóźnionych do pozostałych w rozkładzie dniowym')

[Więcej ustwień wykresów](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.plot.html)

In [ ]:
delays_by_dayOfWeek['flights_count']= (delays_by_dayOfWeek[False]+delays_by_dayOfWeek[True])
delays_by_dayOfWeek[:5]

In [ ]:
delays_by_dayOfWeek['proportion_delayed'] = delays_by_dayOfWeek[True] / delays_by_dayOfWeek['flights_count']
delays_by_dayOfWeek[:4]

In [ ]:
delays_by_dayOfWeek.sort_values('proportion_delayed', ascending=False)

In [ ]:
monday = df[df['DayOfWeek'] == 'Monday']['ArrDelay']

In [ ]:
monday.describe()

In [ ]:
import numpy as np
bin_values = np.arange(start=-50, stop=200, step=10)
print(bin_values)

In [ ]:
monday.hist(bins=bin_values, figsize=[14,6])

In [ ]:
monday.plot(kind='box', figsize=[14,6])

In [ ]:
from revoscalepy import rx_lin_mod

In [ ]:
formula = "ArrDelay ~ DayOfWeek"

revoscalepy.rx_lin_mod(formula, data, pweights=None, fweights=None, cube: bool = False, 
                       cube_predictions: bool = False, row_selection: str = None, 
                       transforms: dict = None, transform_objects: dict = None, 
                       transform_function: typing.Union[str, <built-in function callable>] = None, 
                       transform_variables: list = None, transform_packages: list = None, 
                       transform_environment=None, drop_first: bool = False, 
                       drop_main: bool = True, cov_coef: bool = False, cov_data: bool = False, 
                       blocks_per_read: int = 1, report_progress: int = None, verbose: int = 0, 
                       compute_context=None, **kwargs)
Description
Fit linear models on small or large data.


In [ ]:
lin_mod = rx_lin_mod(formula, data = data_source, compute_context = cc,cube=True)

In [ ]:
print(lin_mod)

In [ ]:
print(lin_mod.summary())